In [3]:
import pandas as pd

train_path = r'C:\Users\DELL\Downloads\train_LZdllcl (1).csv'
test_path = r'C:\Users\DELL\Downloads\test_2umaH9m (1).csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_info = train_df.info()
test_info = test_df.info()
train_head = train_df.head()
test_head = test_df.head()

train_info, test_info, train_head, test_head



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB
<class 'panda

(None,
 None,
    employee_id         department     region         education gender  \
 0        65438  Sales & Marketing   region_7  Master's & above      f   
 1        65141         Operations  region_22        Bachelor's      m   
 2         7513  Sales & Marketing  region_19        Bachelor's      m   
 3         2542  Sales & Marketing  region_23        Bachelor's      m   
 4        48945         Technology  region_26        Bachelor's      m   
 
   recruitment_channel  no_of_trainings  age  previous_year_rating  \
 0            sourcing                1   35                   5.0   
 1               other                1   30                   5.0   
 2            sourcing                1   34                   3.0   
 3               other                2   39                   1.0   
 4               other                1   45                   3.0   
 
    length_of_service  KPIs_met >80%  awards_won?  avg_training_score  \
 0                  8              1         

In [5]:

from sklearn.preprocessing import LabelEncoder

train_df['education'].fillna(train_df['education'].mode()[0], inplace=True)
test_df['education'].fillna(test_df['education'].mode()[0], inplace=True)

# Fill 'previous_year_rating' with median
train_df['previous_year_rating'].fillna(train_df['previous_year_rating'].median(), inplace=True)
test_df['previous_year_rating'].fillna(test_df['previous_year_rating'].median(), inplace=True)

# Encode categorical variables using Label Encoding
categorical_cols = ['department', 'region', 'education', 'gender', 'recruitment_channel']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    train_df[col] = le.fit_transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    label_encoders[col] = le  # store encoders in case needed later

# Drop unnecessary columns
X_train = train_df.drop(columns=['employee_id', 'is_promoted'])
y_train = train_df['is_promoted']
X_test = test_df.drop(columns=['employee_id'])

# Output the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: (54808, 12)
y_train shape: (54808,)
X_test shape: (23490, 12)


C:\Users\DELL\AppData\Local\Temp\ipykernel_12184\2622008838.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['education'].fillna(train_df['education'].mode()[0], inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12184\2622008838.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always be

In [7]:
#  Modeling using Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# 1. Split train data into training and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# 2. Initialize and train the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf_model.fit(X_tr, y_tr)

# 3. Predict on validation set
y_pred = rf_model.predict(X_val)

# 4. Evaluate performance
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))

print("\nClassification Report:")
print(classification_report(y_val, y_pred))

f1 = f1_score(y_val, y_pred)
print(f"\nF1 Score: {f1:.4f}")


Confusion Matrix:
[[9986   42]
 [ 659  275]]

Classification Report:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     10028
           1       0.87      0.29      0.44       934

    accuracy                           0.94     10962
   macro avg       0.90      0.65      0.70     10962
weighted avg       0.93      0.94      0.92     10962


F1 Score: 0.4396


In [10]:
# Fine-tuning the Random Forest model

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Create the base model
rf = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=3,                # 3-fold cross-validation
                           verbose=2,
                           n_jobs=-1)           # Use all CPU cores

# Fit the model
grid_search.fit(X_train, y_train)

# Best parameters and best model
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# Evaluate on training set (or validation set if you split it)
y_pred_train = best_model.predict(X_train)
print("\nClassification Report (Train Data):")
print(classification_report(y_train, y_pred_train))


Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}

Classification Report (Train Data):
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     50140
           1       0.93      1.00      0.96      4668

    accuracy                           0.99     54808
   macro avg       0.96      0.99      0.98     54808
weighted avg       0.99      0.99      0.99     54808



In [31]:

# Step 5: Predict on test data using the best model
test_predictions = best_model.predict(X_test)

# Create a DataFrame for submission
submission = pd.DataFrame({
    'employee_id': test_df['employee_id'],
    'is_promoted': test_predictions
})

# Save the submission file as CSV
submitt=r'C:\Users\DELL\Downloads\sample_submission_M0L0uXE.csv'
submission.to_csv(submitt, index=False)

# Show the first few rows to verify
submission.head()


,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [33]:
submitt


'C:\\Users\\DELL\\Downloads\\sample_submission_M0L0uXE.csv'